In [1]:
import sys
sys.path.append('../jupyterlab_sparksql/')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

%load_ext magics
%load_ext autoreload
%autoreload 2

<IPython.core.display.Javascript object>

In [3]:
table2 = spark.read.csv('people-2000000.csv', header=True, inferSchema=True).drop('Index')

In [20]:
%%sql table1 
select
    *
from (values
    (0,0.0, '2023-05-31', 'line1\nline2'),
    (0,0.24, '2023-04-01', 'line1\nline2'),
    (0,0.56, '2023-05-01', 'line1\nline2'),
    (1,0.99, '2023-03-01', 'line1\nline2'),
    (1,1.0, '2023-03-01', 'line1\nline2')
      
) as t(id, scale_score, update_time, test)
order by update_time desc

Output()

In [5]:
from pyspark.sql import functions as F

In [7]:
table1.withColumn(
    'tier',
    5 - (F.col('scale_score') * 5 - 1e-4).cast('int')
).show()

+---+-----------+-----------+----+
| id|scale_score|update_time|tier|
+---+-----------+-----------+----+
|  0|       0.00| 2023-05-31|   5|
|  0|       0.56| 2023-05-01|   3|
|  0|       0.24| 2023-04-01|   4|
|  1|       0.99| 2023-03-01|   1|
|  1|       1.00| 2023-03-01|   1|
+---+-----------+-----------+----+



In [8]:
table2.printSchema()

root
 |-- User Id: string (nullable = true)
 |-- First Name: string (nullable = true)
 |-- Last Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Email: string (nullable = true)
 |-- Phone: string (nullable = true)
 |-- Date of birth: date (nullable = true)
 |-- Job Title: string (nullable = true)



In [33]:
%%sql
select
    *
from table2

Output()

In [32]:
%%sql table3
select
    `User Id` as user_id
    , `First Name` as first_name
    , `Last Name` as last_name
    , Sex as sex
    , Email as email
    , Phone as phone
    , `Date of birth` as dob
    , `Job Title` as job_title
from table2

Output()

In [30]:
%%sql
select
    *
    , regexp_replace(job_title, r'\s', '\n') as job_title2
from table3

Output()

In [ ]:
!pip install -U pandas

In [20]:
table2.toPandas()

,User Id,First Name,Last Name,Sex,Email,Phone,Date of birth,Job Title
0,4defE49671cF860,Sydney,Shannon,Male,tvang@example.net,574-440-1423x9799,2020-07-09,Technical brewer
1,F89B87bCf8f210b,Regina,Lin,Male,helen14@example.net,001-273-664-2268x90121,1909-06-20,"Teacher, adult education"
2,Cad6052BDd5DEaf,Pamela,Blake,Female,brent05@example.org,927-880-5785x85266,1964-08-19,Armed forces operational officer
3,e83E46f80f629CD,Dave,Hoffman,Female,munozcraig@example.org,001-147-429-8340x608,2009-02-19,Ship broker
4,60AAc4DcaBcE3b6,Ian,Campos,Female,brownevelyn@example.net,166-126-4390,1997-10-02,Media planner
...,...,...,...,...,...,...,...,...
1999995,AbBE4F3Eb5df843,Susan,Ellison,Male,yward@example.net,855.222.4135x62040,1913-01-05,Research scientist (maths)
1999996,e71af3E8De63CDF,Cole,Sherman,Female,nbarnett@example.org,(130)829-9159x800,1977-04-15,Marketing executive
1999997,fAF0618097acD1B,Ethan,Leonard,Male,brett57@example.com,514.898.2376x3069,1960-08-14,Photographer
1999998,f1FC3346F68DFAb,Jordan,Waller,Female,harveykendra@example.com,6920274656,1911-09-05,Chartered accountant
